In [1]:
from sklearn.inspection import partial_dependence
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.inspection import permutation_importance

from matplotlib import pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
import matplotlib.dates as mdate
import time
import random
from sklearn.ensemble import RandomForestClassifier
random_seed=251
random.seed(random_seed)
np.random.seed(random_seed)
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Times']})
# rc.rcParams['font.size'] = 12
rc('text', usetex=True)

In [2]:
# moving_window=12

df= pd.read_csv('D:/istb_4/ISTB4_fillcombdata2.csv')

def norm(df1,df2):
    df1_norm=(df1-df2.min())/(df2.max()-df2.min())
    return df1_norm

In [3]:

Cl_y = np.copy(df['Chlorine.ppm.'])
Cl_x =df[['pH','Conductivity.uS.','Temperature.C.','ORP.mV.','ISTB4.Sum']]

In [4]:
Cl_y[Cl_y<=0.2]=1
Cl_y[(Cl_y>0.2) & (Cl_y<1) ]=0

df.index=pd.to_datetime(df.Time)

In [5]:
Cl_y[Cl_y==1].shape, Cl_y[Cl_y==0].shape

((45395,), (28109,))

In [ ]:
fig, ax=plt.subplots(nrows=5, ncols=1, sharex= True, figsize=(10,5), dpi=200)
axis=ax.flatten()
xpp=np.arange(0,len(df.index))[27200:27400]
dfpp=df.iloc[27200:27400,:]
axis[0].plot(xpp,dfpp['Chlorine.ppm.'],label='Chlorine',linewidth=0.5,color='k',alpha=1)           

axis[1].plot(xpp,dfpp['ORP.mV.'],label='ORP.mV.',linewidth=0.5, color='k',alpha=1)           

axis[2].plot(xpp,dfpp['pH'],label='pH',linewidth=0.5, color='k',alpha=1)           
axis[3].plot(xpp,dfpp['Temperature.C.'],label='Temperature.C.',linewidth=0.5, color='k',alpha=1)           
axis[4].plot(xpp,dfpp['Conductivity.uS.'],label='Conductivity.uS.',linewidth=0.5, color='k',alpha=1)   


axis[0].set_ylabel('Chlorine')
axis[1].set_ylabel('ORP.mV')
axis[2].set_ylabel('pH')

axis[3].set_ylabel('Temperature.C.')
axis[4].set_ylabel('Conductivity.uS')
plt.grid()

In [ ]:




fig, ax2=plt.subplots(nrows=1, ncols=1, sharex= True, figsize=(8,4), dpi=200)

ax1 = ax2.twinx()


monthFmt = mdate.DateFormatter('%Y-%b')

ax1.bar(df.index, Cl_y, label='label', color='g',alpha=0.01)   

ax1.set_xlabel('Time',fontsize=20)
ax1.tick_params(axis='y')
ax1.xaxis.set_ticks_position('both')
ax1.tick_params(direction='in', labelsize=13)

ax1.yaxis.set_ticks(np.arange(0, 3, 1))   
ax1.set_ylim([0,1])
ax1.get_yaxis().set_visible(False)

ax2.plot(df.index,df['Chlorine.ppm.'],label='Chlorine',linewidth=2, color='k',alpha=1)           
ax2.set_ylabel('Chlorine (ppm)',fontsize=15)
# ax2.tick_params(axis='y')
ax2.xaxis.set_ticks_position('both')
ax2.tick_params(direction='in',labelsize=13)
plt.legend()
# ax2.set_xticklabels(df.index,fontsize=13,rotation=90)
# ax1.set_xticklabels(df.index,fontsize=13,rotation=90)#
# plt.tick_params(labelbottom = False)
# ax1.xaxis.set_major_locator(mdate.MonthLocator(bymonthday=1))
# ax1.xaxis.set_major_formatter(mdate.DateFormatter('%Y-%b'))


# ax_leg = ax1.inset_axes([.15, .15, .15, .15])
# ax_leg.plot([np.nan], label='Chlorine',linewidth=1, color='k',alpha=1)
# ax_leg.bar([np.nan], [np.nan], label='Label', color='g',alpha=0.5)
# ax_leg.legend(fontsize=13,facecolor='white', framealpha=1)
# ax_leg.axis('off')


In [ ]:
"'  shuffle'"
dx =Cl_x[:-1]
dy =Cl_y[1:]


X_traincv,X_test,y_traincv,y_test=train_test_split(dx,dy ,test_size=0.2, random_state=random_seed,stratify=dy)
X_train,X_cv,y_train,y_cv=train_test_split(X_traincv,y_traincv ,test_size=0.2, random_state=random_seed,stratify=y_traincv)

X_train.shape, y_train.shape,X_test.shape, y_test.shape


In [ ]:
# X_train =Cl_x[:40000]
# y_train =Cl_y[1:40001]

# X_test =Cl_x[40000:50000]
# y_test =Cl_y[40001:50001]
# X_train.shape, y_train.shape,X_test.shape, y_test.shape

In [ ]:
rfm = RandomForestClassifier(n_jobs=-1,n_estimators=800,random_state=random_seed,max_features=4,min_samples_leaf=20)
rfm.fit(X_train,y_train)
vari_imp_mn=rfm.feature_importances_
yhte= rfm.predict(X_test)
yhtr=rfm.predict(X_train)
# yhcv=rfm.predict(X_cv)

In [ ]:
accuracy_score(y_test, yhte),accuracy_score(y_train, yhtr)#,accuracy_score(y_cv, yhcv)

In [ ]:
accuracy_score(y_test, yhte),accuracy_score(y_train, yhtr)#,accuracy_score(y_cv, yhcv)

In [ ]:
confusion_matrix(y_test, yhte,labels=[1,0],normalize='all')

# PERMUATATION

In [ ]:
from sklearn.inspection import permutation_importance
resultss = permutation_importance(rfm, X_train, y_train, n_repeats=10,
                                random_state=0)


In [ ]:
variables_name=['pH', 'Conductivity', 'Temperature', 'ORP', 'Occupancy']

forest_importances = pd.DataFrame(data=resultss.importances_mean, index=variables_name, columns=['permute'])
kkk_sort=forest_importances.sort_values('permute',ascending=False)


In [ ]:
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Times']})
# rc.rcParams['font.size'] = 12
rc('text', usetex=True)
txt_size=13
label_size=10
fig, ax=plt.subplots(nrows=1, ncols=1, sharey= True, figsize=(5,3), dpi=200)

ax.bar(kkk_sort.index,kkk_sort['permute']/kkk_sort['permute'].sum())
ax.set_ylabel('Variable importance',fontsize=txt_size)
ax.xaxis.set_ticks_position('both')
ax.tick_params(direction='in',labelsize=label_size)
ax.yaxis.set_ticks_position('both')
ax.set_yticks(ticks=np.arange(0,0.45,.1),labels=['0','0.1','0.2','0.3','0.4'])
# ax.set_xticks(variables_name, fontsize=label_size)
plt.show()

# PDP

In [ ]:
from sklearn.inspection import PartialDependenceDisplay
PartialDependenceDisplay.from_estimator(rfm,X_train, [0, 1, 2,3,4],kind='both')

In [ ]:
results_value_pdp=[]
results_avg_pdp=[]
for i_inde in range (5):
    results = partial_dependence(rfm,X_train, [i_inde],percentiles=[0, 1],grid_resolution=500)
    results_value_pdp.append(np.array(results["values"]).flatten())
    results_avg_pdp.append(np.array(results["average"]).flatten())

In [ ]:
txt_size=12
label_size=10
fig, ax=plt.subplots(nrows=2, ncols=3, sharey= True, figsize=(10,5), dpi=200)
features = [0, 1, 2,3,4]
variables_name=['pH', 'Conductivity (uS/cm)', 'Temperature ($^o$ C)', 'ORP (mV)', 'Occupancy']
axes=ax.flatten()
for i_inde in range (5):
    axes[i_inde].plot(results_value_pdp[i_inde],results_avg_pdp[i_inde],'k')
    axes[i_inde].tick_params(axis='both', direction='in',labelsize=label_size) 
    axes[i_inde].set_xlabel(variables_name[i_inde])

    axes[i_inde].set_yticks(ticks=np.arange(0,1.2, .2),labels=['0','0.2','0.4','0.6','0.8','1'])
    sns.rugplot(X_train.iloc[:,i_inde],ax=axes[i_inde],c='k', alpha=0.03,height=0.03)
axes[0].set_ylabel('Predicted Probility of label 1')
axes[3].set_ylabel('Predicted Probility of label 1')
fig.delaxes(axes[-1])
fig.tight_layout() 
plt.show()

# 8 hours results

In [ ]:
step_list=list(range(0,588,12))
step_list[0]=1
r2_te_list=[]
r2_tr_list=[]
mse_tr=[]
mse_te=[]
for step in step_list:
    # step=1
    X_train =Cl_x[:40000]
    y_train =Cl_y[step:40000+step]

    X_test =Cl_x[40000:len(Cl_x)-step]
    y_test =Cl_y[40000+step:]
    rfm = RandomForestClassifier(n_jobs=-1,n_estimators=800,random_state=random_seed,max_features=4,min_samples_leaf=40)
    rfm.fit(X_train,y_train)
    vari_imp_mn=rfm.feature_importances_
    hpa_yh= rfm.predict(X_train)
    yhte= rfm.predict(X_test)

    yhtr=rfm.predict(X_train)
    r2_tr_list.append(r2_score(y_train,yhtr))
    r2_te_list.append(r2_score(y_test,yhte))
    mse_tr.append(mean_squared_error(y_train,yhtr))
    mse_te.append(mean_squared_error(y_test,yhte))

In [6]:
"' N time step ahead random shuffle'"
step_list=list(range(1,96))
var_im=[]
r2_te_list=[]
r2_tr_list=[]
con_mat=[]
pmut_im=[]


y_te_pred_all=[]
y_te_all=[]

x_tr_all=[]
x_te_all=[]
y_tr_all=[]


for step in step_list:
    # step=1
    Cl_xstep1=Cl_x[:len(Cl_x)-step]
    Cl_ystep1=Cl_y[step:]
    X_train,X_test,y_train,y_test=train_test_split(Cl_xstep1,Cl_ystep1 ,test_size=0.2, random_state=random_seed,stratify=Cl_ystep1)
    rfm = RandomForestClassifier(n_jobs=-1,n_estimators=800,random_state=random_seed,max_features=4,min_samples_leaf=20)
    rfm.fit(X_train,y_train)
    vari_imp_mn=rfm.feature_importances_
    yhte= rfm.predict(X_test)
    # yhtr=rfm.predict(X_train)
    y_te_pred_all.append(yhte)
    x_tr_all.append(X_train)
    y_tr_all.append(y_train)
    x_te_all.append(X_test)
    y_te_all.append(y_test)
    # resultss = permutation_importance(rfm, X_train, y_train, n_repeats=10,
    #                             random_state=0)
    # pmut_im.append(resultss.importances_mean)
    # r2_tr_list.append(accuracy_score(y_train, yhtr))
    # r2_te_list.append(accuracy_score(y_test, yhte))
    # con_mat.append(confusion_matrix(y_test, yhte,labels=[0,1],normalize='all'))
    # var_im.append(vari_imp_mn)

In [41]:
kkkk_test=np.load('D:/istb_4/data/rf_classifier/step_95.npz')

In [ ]:
kkkk_test.files

In [ ]:
kkkk_test['te']

In [40]:

for i in range (len(y_te_pred_all)):
    x_tr_all[i]['Chlorine_label']=y_tr_all[i]
    x_te_all[i]['Chlorine_label']=y_te_all[i]
    x_te_all[i]['Chlorine_pred_label']=y_te_pred_all[i]
    np.savez('D:/istb_4/data/rf_classifier/step_'+ str(i+1)+'.npz',te=x_te_all[i],tr=x_tr_all[i])

In [ ]:
np.save('D:/istb_4/rf_classifier/step_ahead_random_shuffle_ACC_tr_imb_8hr.npy',r2_tr_list)
np.save('D:/istb_4/rf_classifier/step_ahead_random_shuffle_ACC_te_imb_8hr.npy',r2_te_list)
np.save('D:/istb_4/rf_classifier/step_ahead_random_shuffle_con_mat_te_imb_8hr.npy',con_mat)
np.save('D:/istb_4/rf_classifier/step_ahead_random_shuffle_rfc_vari_imp_mn_8hr.npy',vari_imp_mn)


In [ ]:
np.save('D:/istb_4/rf_classifier/step_ahead_random_shuffle_rfc_vari_imp_permutation_8hr.npy',pmut_im)


# Variable imp

In [ ]:
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Times']})
# rc.rcParams['font.size'] = 12
rc('text', usetex=True)
txt_size=13
label_size=10
mean_var_im=np.array(pmut_im).mean(axis=0)
variables_name=['pH', 'Conductivity', 'Temperature', 'ORP', 'Occupancy']

varplot=pd.DataFrame(data= mean_var_im, index=variables_name, columns=['gini'])
dfx = varplot.sort_values('gini',ascending=False)
fig, ax=plt.subplots(nrows=1, ncols=1, sharey= True, figsize=(5,3), dpi=200)

ax.bar(dfx.index,dfx['gini']/dfx['gini'].sum())
ax.set_ylabel('Variable importance',fontsize=txt_size)
ax.xaxis.set_ticks_position('both')
ax.tick_params(direction='in',labelsize=label_size)
ax.yaxis.set_ticks_position('both')
# ax.set_xticks(variables_name, fontsize=label_size)
plt.show()


# accuracy vs time step

In [ ]:
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Times']})
# rc.rcParams['font.size'] = 12
rc('text', usetex=True)
txt_size=13
label_size=10
fig, ax=plt.subplots(nrows=1, ncols=1, sharey= True, figsize=(4,3), dpi=200)
ax.plot(np.array(step_list)*5/60,np.array(r2_tr_list)*100,'r', linestyle='--', label='RF classification Train')
ax.plot(np.array(step_list)*5/60,np.array(r2_te_list)*100,'b',linestyle='--',markersize=3,label='RF classification Test')
# ax.set_ylim([95,100])
ax.legend()
ax.set_xlabel('Time ahead (hr)',fontsize=txt_size)
ax.set_ylabel('Accuracy',fontsize=txt_size)
ax.tick_params(axis='both', direction='in',labelsize=label_size) 
plt.show()

In [ ]:
con_mat[30]